In [1]:
import numpy as np
import sys
sys.path.append("../")
from algorithm.koopmanlib.dictionary import PsiNN

In [2]:
basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=22)

In [3]:
from duffing_oscillator import duffing_oscillator
from scipy.integrate import solve_ivp

# Parameters
delta = 0.5
alpha = -1
beta = 1

# Sample N=1000 initial points, with uniform distribution [-2, 2].
N = 1000
np.random.seed(39)
sample_data_x0 = (np.random.rand(N, 2) * 4) - 2

tspan = [0, 2.5]
nstep = 11
data = np.zeros((N, 2, nstep))

t_eval = np.linspace(tspan[0], tspan[1], nstep)

for idx, x0 in enumerate(sample_data_x0):
    sol = solve_ivp(duffing_oscillator, tspan, x0, t_eval=t_eval, args=(delta, alpha, beta))
    data[idx, 0, :] = sol.y[0]
    data[idx, 1, :] = sol.y[1]

# Pairwise shifted data X and Y
X = data[:, :, :-1] # shape: (1000, 2, 10)
Y = data[:, :, 1:]

# Transpose and reshape the data.
X = np.transpose(X, (0, 2, 1)).reshape(N * (nstep-1), 2) # shape: (10000, 2)
Y = np.transpose(Y, (0, 2, 1)).reshape(N * (nstep-1), 2)

In [4]:
len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

In [5]:
data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

In [6]:
# Using solver0
from algorithm.koopmanlib.solver_0_updated import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=3000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 173ms/step - loss: 0.0023 - val_loss: 2.2824e-04
Epoch 2/2
2/2 [==============================] - 0s 35ms/step - loss: 0.0016 - val_loss: 0.0047
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 57ms/step - loss: 1.0461e-04 - val_loss: 2.1721e-04
Epoch 2/2
2/2 [==============================] - 0s 36ms/step - loss: 1.8263e-04 - val_loss: 1.3079e-04
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 56ms/step - loss: 8.9208e-05 - val_loss: 1.4598e-04
Epoch 2/2
2/2 [==============================] - 0s 35ms/step - loss: 1.2398e-04 - val_loss: 1.1089e-04
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 54ms/step - loss: 8.7136e-05 - val_loss: 1.1949e-04
Epoch 2/2
2/2 [==============================] - 0s 48ms/step - loss: 9.9027e-05 - val_loss: 1.0458e-04
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 53ms/step - los

In [7]:
# Results from solver0
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs0 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_0_outputs.npy", resDMD_DL_outputs0, allow_pickle=True)

print(resDMD_DL_outputs0['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs0['resDMD_DL_outputs']['efuns'].shape)

[ 1.01288077e+00+0.00000000e+00j  9.99998601e-01+0.00000000e+00j
  8.33722757e-01+1.61171451e-01j  8.33722757e-01-1.61171451e-01j
  7.78575550e-01+4.09716729e-01j  7.78575550e-01-4.09716729e-01j
  6.51436332e-01-2.58137454e-01j  6.51436332e-01+2.58137454e-01j
  4.09500138e-01+5.03979665e-01j  4.09500138e-01-5.03979665e-01j
  7.76790528e-02+4.73624365e-02j  7.76790528e-02-4.73624365e-02j
  4.74638742e-02-6.51144677e-01j  4.74638742e-02+6.51144677e-01j
  3.88019334e-02+3.00991684e-02j  3.88019334e-02-3.00991684e-02j
  1.36891625e-03+7.94528783e-05j  1.36891625e-03-7.94528783e-05j
  5.01087849e-04+2.59462335e-04j  5.01087849e-04-2.59462335e-04j
 -1.18487411e-04+0.00000000e+00j -1.31581486e-04-1.40607135e-04j
 -1.31581486e-04+1.40607135e-04j -1.72792326e-02+2.03422840e-02j
 -1.72792326e-02-2.03422840e-02j]
(10000, 25)


In [8]:
# Using solver1
from algorithm.koopmanlib.solver_1_updated import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=3000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 8s 1s/step - loss: 0.4282 - val_loss: 0.4517
Epoch 2/2
2/2 [==============================] - 0s 102ms/step - loss: 0.4339 - val_loss: 0.4317
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 111ms/step - loss: 0.4238 - val_loss: 0.4157
Epoch 2/2
2/2 [==============================] - 0s 87ms/step - loss: 0.3995 - val_loss: 0.4286
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 110ms/step - loss: 0.4008 - val_loss: 0.4297
Epoch 2/2
2/2 [==============================] - 0s 87ms/step - loss: 0.4031 - val_loss: 0.4263
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 113ms/step - loss: 0.4048 - val_loss: 0.4165
Epoch 2/2
2/2 [==============================] - 0s 91ms/step - loss: 0.4128 - val_loss: 0.4198
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 118ms/step - loss: 0.4211 - val_loss: 0.4256
Epoch 2/2
2/2 [======

In [9]:
# Results from solver1
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs1 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_1_outputs.npy", resDMD_DL_outputs1, allow_pickle=True)

print(resDMD_DL_outputs1['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs1['resDMD_DL_outputs']['efuns'].shape)

[1.00446422+0.j         1.00000114+0.j         0.8614515 +0.j
 0.82912841-0.14173137j 0.82912841+0.14173137j 0.82616986-0.36721784j
 0.82616986+0.36721784j 0.78803974-0.32149464j 0.78803974+0.32149464j
 0.72245587+0.j         0.69840416-0.2735262j  0.69840416+0.2735262j
 0.48967047-0.48266023j 0.48967047+0.48266023j 0.40006413+0.47415417j
 0.40006413-0.47415417j 0.39623351+0.j         0.2036226 -0.1376253j
 0.2036226 +0.1376253j  0.15794205-0.59895357j 0.15794205+0.59895357j
 0.09144583+0.j         0.00477258-0.0759844j  0.00477258+0.0759844j
 0.00209648+0.j        ]
(10000, 25)


In [10]:
# Using solver2
from algorithm.koopmanlib.solver_2_updated import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=3000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 2s 357ms/step - loss: 430453.4018 - val_loss: 280237.7500
Epoch 2/2
2/2 [==============================] - 0s 77ms/step - loss: 377121.4420 - val_loss: 251014.5781
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 94ms/step - loss: 339150.2500 - val_loss: 228054.0000
Epoch 2/2
2/2 [==============================] - 0s 74ms/step - loss: 306745.7321 - val_loss: 209695.5000
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 103ms/step - loss: 280660.3482 - val_loss: 194676.0000
Epoch 2/2
2/2 [==============================] - 0s 62ms/step - loss: 259710.6741 - val_loss: 182019.8438
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 103ms/step - loss: 241088.0938 - val_loss: 170486.5312
Epoch 2/2
2/2 [==============================] - 0s 87ms/step - loss: 228001.2857 - val_loss: 166605.6406
number of the outer loop: 3
Epoch 1/2
2/2 [======================

In [11]:
# Results from solver2
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs2 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_2_outputs.npy", resDMD_DL_outputs2, allow_pickle=True)

print(resDMD_DL_outputs2['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs2['resDMD_DL_outputs']['efuns'].shape)

[ 1.00003786+0.j          0.98275628+0.j          0.81191075+0.18502234j
  0.81191075-0.18502234j  0.77136101+0.j          0.7435516 +0.34126418j
  0.7435516 -0.34126418j  0.62049928-0.40392684j  0.62049928+0.40392684j
  0.51792316+0.16849615j  0.51792316-0.16849615j  0.46601568+0.j
  0.4020027 +0.38381424j  0.4020027 -0.38381424j  0.18811347+0.j
  0.13032641+0.25871814j  0.13032641-0.25871814j  0.07704621-0.52892193j
  0.07704621+0.52892193j  0.02894321+0.j          0.02729046+0.j
  0.00264076+0.j          0.00120823+0.j         -0.00143223+0.j
 -0.00664268+0.j        ]
(10000, 25)


In [12]:
# Using solver3
from algorithm.koopmanlib.solver_3_updated import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=100,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 218ms/step - loss: 0.0044 - val_loss: 0.0012
Epoch 2/2
2/2 [==============================] - 0s 59ms/step - loss: 0.0036 - val_loss: 0.0091
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 79ms/step - loss: 1.9864e-04 - val_loss: 6.0860e-04
Epoch 2/2
2/2 [==============================] - 0s 57ms/step - loss: 4.5955e-04 - val_loss: 3.5799e-04
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 69ms/step - loss: 1.5027e-04 - val_loss: 4.0733e-04
Epoch 2/2
2/2 [==============================] - 0s 48ms/step - loss: 3.4643e-04 - val_loss: 1.7091e-04
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 67ms/step - loss: 1.2756e-04 - val_loss: 3.6442e-04
Epoch 2/2
2/2 [==============================] - 0s 47ms/step - loss: 2.9363e-04 - val_loss: 1.0671e-04
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 67ms/step - loss: 1

In [13]:
# Results from solver3
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs3 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_3_outputs.npy", resDMD_DL_outputs3, allow_pickle=True)

print(resDMD_DL_outputs3['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs3['resDMD_DL_outputs']['efuns'].shape)

[ 1.00001005+0.j          0.9832197 +0.j          0.81177625+0.19358096j
  0.81177625-0.19358096j  0.79119945+0.j          0.74207328-0.35366571j
  0.74207328+0.35366571j  0.60491203+0.44625551j  0.60491203-0.44625551j
  0.57937851+0.j          0.45620778+0.31097536j  0.45620778-0.31097536j
  0.4286684 +0.0844741j   0.4286684 -0.0844741j   0.12292096+0.22273592j
  0.12292096-0.22273592j  0.10414487+0.j          0.07238552-0.56106345j
  0.07238552+0.56106345j  0.03226451+0.00517532j  0.03226451-0.00517532j
  0.00362147+0.j          0.00100424+0.00078589j  0.00100424-0.00078589j
 -0.01332333+0.j        ]
(10000, 25)


In [14]:
# Using solver4
from algorithm.koopmanlib.solver_4_updated import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=3000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 3s 546ms/step - loss: 0.2391 - val_loss: 0.1988
Epoch 2/2
2/2 [==============================] - 0s 64ms/step - loss: 0.2134 - val_loss: 0.1786
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 82ms/step - loss: 0.1906 - val_loss: 0.1496
Epoch 2/2
2/2 [==============================] - 0s 68ms/step - loss: 0.1838 - val_loss: 0.1360
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 91ms/step - loss: 0.1544 - val_loss: 0.1305
Epoch 2/2
2/2 [==============================] - 0s 73ms/step - loss: 0.1460 - val_loss: 0.1256
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 90ms/step - loss: 0.1441 - val_loss: 0.1188
Epoch 2/2
2/2 [==============================] - 0s 66ms/step - loss: 0.1381 - val_loss: 0.1155
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 88ms/step - loss: 0.1321 - val_loss: 0.1173
Epoch 2/2
2/2 [========

In [15]:
# Results from solver4
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs4 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_4_outputs.npy", resDMD_DL_outputs4, allow_pickle=True)

print(resDMD_DL_outputs4['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs4['resDMD_DL_outputs']['efuns'].shape)

[ 1.00005867e+00+0.00000000e+00j  9.55428120e-01+0.00000000e+00j
  8.31254804e-01+0.00000000e+00j  7.91600547e-01+2.08344630e-01j
  7.91600547e-01-2.08344630e-01j  7.75367287e-01+3.96647917e-01j
  7.75367287e-01-3.96647917e-01j  7.43501276e-01+0.00000000e+00j
  5.27311824e-01+5.60801784e-01j  5.27311824e-01-5.60801784e-01j
  1.39258975e-01-2.51679694e-02j  1.39258975e-01+2.51679694e-02j
  6.91770748e-02+7.04905827e-02j  6.91770748e-02-7.04905827e-02j
  5.34778907e-02-1.26127784e-01j  5.34778907e-02+1.26127784e-01j
  4.52720195e-02+0.00000000e+00j  2.89574737e-02+3.02630147e-02j
  2.89574737e-02-3.02630147e-02j  2.35740122e-02+7.31757505e-03j
  2.35740122e-02-7.31757505e-03j  1.58766452e-04+6.32008409e-05j
  1.58766452e-04-6.32008409e-05j -4.40896097e-04+2.19747474e-04j
 -4.40896097e-04-2.19747474e-04j]
(10000, 25)
